Here are the steps involved in this:



1.   Read a pdf from a file location
2.   Convert those pdf into chunks
3.   Store those chunks as embeddings using any embedding model
4.   Read a question or prompt from the user

    - Convert it into embedding too
    - Run Faiss on the prompt with the existing embedding database
    - Pass the k-nearest embeddings as a context through the prompt template
5. Pass the context and the question to LLM and get the response.

In [ ]:
!pip install openai
!pip install faiss-cpu
!pip install langchain
!pip install python-dotenv
!pip install pypdf2
!pip install langchain_openai
!pip install sentence-transformers
!pip install langchain_community
!pip install streamlit

In [132]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import PyPDF2 as pypdf
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

from langchain_community.llms import HuggingFaceHub
import streamlit as st



load_dotenv()

os.chdir('/content/drive/MyDrive/Colab Notebooks/LangchainRAGQA')
client = OpenAI(api_key = os.environ.get('OPENAI_API_KEY'))

### Split PDF to chunks

In [135]:
def pdf_to_faiss(pdf_location, chunk_size=800, chunk_overlap=100):
    #pdf_location = '/content/drive/MyDrive/Colab Notebooks/LangchainRAGQA/budget_speech.pdf'
    pdf = pypdf.PdfReader(pdf_location)
    full_text = ''
    for i, content in enumerate(pdf.pages):
        raw_text = content.extract_text()
        full_text += raw_text
    text_splits = RecursiveCharacterTextSplitter(separators='\n', chunk_size = 800, chunk_overlap = 100, length_function=len).split_text(full_text)
    embeddings = HuggingFaceEmbeddings()
    db = FAISS.from_texts(text_splits, embeddings)
    return db

In [158]:
docs=''
def answer(db, question):
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1" #"google/flan-t5-base"
    model = HuggingFaceHub(
        repo_id=repo_id, model_kwargs={"temperature": 0.7, "max_length": 5000}
    )
    docs = db.similarity_search(question, k=10)
    prompt = """Answer the following QUESTION based on the CONTEXT
    given. If you do not know the answer and the CONTEXT doesn't
    contain the answer truthfully say "I don't know"

        CONTEXT:{context}
        QUESTION:{question}
        ANSWER:
        """

    prompt_template = PromptTemplate(
        input_variables=["context", "question"],
        template=prompt,
    )
    chain = LLMChain(llm=model, prompt=prompt_template)
    return prompt, docs, chain.run(context = docs, question=question)

In [148]:
#db = pdf_to_faiss('/content/drive/MyDrive/Colab Notebooks/LangchainRAGQA/budget_speech.pdf')

In [160]:
ask = "Can you tell the impact of covid on indian budget?"
prompt, docs, ans = answer(db, ask)
print(ans)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)



        Based on the provided context, the Revised Estimate of the fiscal deficit for the year 2022-23 is 6.4 per cent of GDP, adhering to the Budget Estimate. This is despite the massive slowdown globally caused by Covid-19 and a war. Therefore, the Covid-19 pandemic has had an impact on the Indian budget, leading to a higher fiscal deficit. However, the budget for the


In [157]:
ans

'\n        Based on the provided context, the Revised Estimate of the fiscal deficit for the year 2022-23 is 6.4 per cent of GDP, adhering to the Budget Estimate. This is despite the massive slowdown globally caused by Covid-19 and a war. Therefore, the Covid-19 pandemic has had an impact on the Indian budget, leading to a higher fiscal deficit. However, the budget for the'

In [161]:
def main():
    st.set_page_config(layout="wide")
    st.title("Custom Summarization App")
    user_prompt = st.text_input("Enter the user prompt")
    pdf_file_path = st.text_input("Enter the pdf file path")
    if pdf_file_path != "":
        db = pdf_to_faiss(pdf_file_path)
        st.write("Pdf was loaded successfully")
        if st.button("Summarize"):
            result = answer(db,user_prompt)
            st.write("Here is the answer to your question:")
            st.write(summary)

In [162]:
main()